# 🎯 第一章 练习 1: 潜在结果框架 (Potential Outcomes Framework)

---

## 欢迎来到因果推断的世界！

想象一下这个场景：

> 你是一家电商公司的数据分析师。老板问你："我们上周给 100 万用户发了优惠券，销售额涨了 500 万。那优惠券到底值不值得发？"

你可能会想：这不简单吗？发了优惠券销售涨了，那就是优惠券的功劳呗！

但是...等等... 🤔

- 如果不发优惠券，销售额会不会也涨？（也许是节假日效应）
- 收到优惠券的用户本来就更活跃吗？（也许运营挑了高价值用户发）
- 500 万的增长真的全是优惠券带来的吗？

这些问题的本质是：**我们想知道优惠券的「真实效果」，但我们只能看到「发了优惠券之后发生了什么」**。

这就是因果推断要解决的核心问题！

---

## 📚 本节学习目标

1. 理解什么是 Y(0), Y(1), ITE, ATE
2. 理解「反事实」(counterfactual) 的含义
3. 理解为什么简单比较会出错
4. 理解随机实验为什么有效

In [ ]:
# 首先导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文显示和绘图风格
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！让我们开始吧！")

---

## 🌟 Part 1: 什么是「潜在结果」？

### 一个生动的例子

假设你感冒了，面前有一粒药💊。你在想：吃还是不吃？

如果我们有上帝视角，可以看到两个平行宇宙：

| 平行宇宙 | 你的选择 | 3天后你的状态 |
|---------|---------|-------------|
| 宇宙 A 🌍 | 吃药 | 康复了 ✅ |
| 宇宙 B 🌎 | 不吃药 | 还在难受 😷 |

这两个结果就是**潜在结果 (Potential Outcomes)**：

- **Y(1)**: 吃药后的结果（治疗后的潜在结果）
- **Y(0)**: 不吃药的结果（未治疗的潜在结果）

药物对你的**个体处理效应 (Individual Treatment Effect, ITE)** 就是：

$$\text{ITE} = Y(1) - Y(0) = \text{康复} - \text{难受} = \text{药物让你好起来了！}$$

### 残酷的现实：反事实问题

但问题来了...在现实中，你只能选择一个！

- 如果你吃了药，你永远不知道「不吃药会怎样」
- 如果你没吃药，你永远不知道「吃了药会怎样」

**我们永远只能观测到一个潜在结果，另一个永远是「反事实」(counterfactual)**。

这就是因果推断的**根本问题 (Fundamental Problem of Causal Inference)**！

### 🧮 数学符号表示

让我们更正式地定义：

对于个体 $i$：

| 符号 | 含义 | 例子 |
|-----|------|------|
| $T_i$ | 处理状态 (0或1) | 是否吃药 |
| $Y_i(0)$ | 未处理时的潜在结果 | 不吃药时的健康状态 |
| $Y_i(1)$ | 处理时的潜在结果 | 吃药后的健康状态 |
| $Y_i$ | 实际观测到的结果 | 你实际的健康状态 |

观测结果和潜在结果的关系：

$$Y_i = T_i \cdot Y_i(1) + (1 - T_i) \cdot Y_i(0)$$

翻译成人话：
- 如果你吃了药 ($T_i = 1$)，我们观测到 $Y_i(1)$
- 如果你没吃药 ($T_i = 0$)，我们观测到 $Y_i(0)$

---

## 🔬 Part 2: 动手生成数据

让我们扮演上帝，生成一个「完美」的数据集，在这个数据集中我们知道每个人的**两个**潜在结果！

### 场景设定

想象我们在研究「学习时间」对「考试成绩」的影响：

- **X**: 学生的基础能力（智力、学习习惯等）
- **T**: 是否参加了考前补习班（1=参加，0=没参加）
- **Y(0)**: 不参加补习班的成绩
- **Y(1)**: 参加补习班的成绩

假设真实的数据生成过程 (DGP) 是：

$$Y(0) = 50 + 5 \cdot X + \epsilon$$
$$Y(1) = 60 + 5 \cdot X + \epsilon$$

注意看：补习班让每个人的成绩提高了 10 分！所以 **真实 ATE = 10**。

In [ ]:
def generate_potential_outcomes(n: int = 100, seed: int = 42) -> pd.DataFrame:
    """
    生成带有潜在结果的模拟数据
    
    我们扮演上帝，知道每个人的两种潜在结果！
    真实 ATE = 10（补习班的真实效果）
    """
    np.random.seed(seed)
    
    # 生成学生的基础能力 X（标准正态分布）
    X = np.random.randn(n)
    
    # 生成潜在结果
    # Y(0) = 50 + 5*X + noise，噪声服从 N(0, 5)
    # Y(1) = 60 + 5*X + noise
    noise = np.random.randn(n) * 5  # 已经帮你生成好噪声了
    Y0 = 50 + 5 * X + noise  # 不参加补习班的成绩
    Y1 = 60 + 5 * X + noise  # 参加补习班的成绩
    
    # 随机分配处理 T（50% 的学生参加补习班）
    T = np.random.binomial(1, 0.5, n)
    
    # 计算观测结果 Y
    # 记住：Y = T * Y(1) + (1-T) * Y(0)
    Y = T * Y1 + (1 - T) * Y0
    
    # 计算个体处理效应 ITE = Y(1) - Y(0)
    ITE = Y1 - Y0
    
    return pd.DataFrame({
        'X': X,         # 基础能力
        'T': T,         # 是否参加补习班
        'Y0': Y0,       # 潜在结果：不参加
        'Y1': Y1,       # 潜在结果：参加
        'Y': Y,         # 观测结果
        'ITE': ITE      # 个体处理效应
    })

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def generate_potential_outcomes(n: int = 100, seed: int = 42) -> pd.DataFrame:
    """
    生成带有潜在结果的模拟数据
    
    我们扮演上帝，知道每个人的两种潜在结果！
    真实 ATE = 10（补习班的真实效果）
    """
    np.random.seed(seed)
    
    # TODO 1: 生成学生的基础能力 X（标准正态分布）
    X = np.random.randn(n)
    
    # TODO 2: 生成潜在结果
    noise = np.random.randn(n) * 5
    Y0 = 50 + 5 * X + noise  # 不参加补习班的成绩
    Y1 = 60 + 5 * X + noise  # 参加补习班的成绩
    
    # TODO 3: 随机分配处理 T（50% 的学生参加补习班）
    T = np.random.binomial(1, 0.5, n)
    
    # TODO 4: 计算观测结果 Y
    Y = T * Y1 + (1 - T) * Y0
    
    # TODO 5: 计算个体处理效应 ITE = Y(1) - Y(0)
    ITE = Y1 - Y0
    
    return pd.DataFrame({
        'X': X,
        'T': T,
        'Y0': Y0,
        'Y1': Y1,
        'Y': Y,
        'ITE': ITE
    })
```

**解释**：
- **X**: 使用 `np.random.randn(n)` 生成标准正态分布的基础能力
- **Y0, Y1**: 根据数据生成过程 Y(0) = 50 + 5X + ε 和 Y(1) = 60 + 5X + ε
- **T**: 使用二项分布 `np.random.binomial(1, 0.5, n)` 实现 50% 随机分配
- **Y**: 观测结果是潜在结果的「切换」公式 Y = T·Y(1) + (1-T)·Y(0)
- **ITE**: 个体效应就是两个潜在结果的差值
</details>

In [ ]:
# 测试你的代码
df = generate_potential_outcomes(n=200)

if df['X'] is not None:
    print("🎉 数据生成成功！")
    print(f"\n样本量: {len(df)}")
    print(f"参加补习班的学生: {df['T'].sum()} ({df['T'].mean()*100:.1f}%)")
    print(f"\n数据预览:")
    display(df.head(10))
else:
    print("❌ 请完成 TODO 部分！")

### 📊 可视化潜在结果

让我们看看这些潜在结果长什么样：

In [ ]:
if df['X'] is not None:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 图1: 两种潜在结果 vs X
    ax1 = axes[0]
    ax1.scatter(df['X'], df['Y0'], alpha=0.5, label='Y(0): 不参加补习', c='blue')
    ax1.scatter(df['X'], df['Y1'], alpha=0.5, label='Y(1): 参加补习', c='red')
    ax1.set_xlabel('基础能力 X')
    ax1.set_ylabel('成绩')
    ax1.set_title('两种潜在结果（上帝视角）')
    ax1.legend()
    
    # 图2: ITE 分布
    ax2 = axes[1]
    ax2.hist(df['ITE'], bins=30, edgecolor='black', alpha=0.7, color='green')
    ax2.axvline(df['ITE'].mean(), color='red', linestyle='--', label=f'平均 ITE = {df["ITE"].mean():.2f}')
    ax2.set_xlabel('个体处理效应 (ITE)')
    ax2.set_ylabel('频数')
    ax2.set_title('ITE 分布')
    ax2.legend()
    
    # 图3: 实际观测到的结果
    ax3 = axes[2]
    treated = df[df['T'] == 1]
    control = df[df['T'] == 0]
    ax3.scatter(treated['X'], treated['Y'], alpha=0.5, label='参加补习(观测)', c='red', marker='o')
    ax3.scatter(control['X'], control['Y'], alpha=0.5, label='未参加(观测)', c='blue', marker='s')
    ax3.set_xlabel('基础能力 X')
    ax3.set_ylabel('成绩')
    ax3.set_title('实际观测到的数据')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🔍 关键发现:")
    print(f"   - 真实 ATE (我们设定的): 10")
    print(f"   - 从数据计算的平均 ITE: {df['ITE'].mean():.2f}")
    print(f"   - ITE 的标准差: {df['ITE'].std():.2f}")

---

## 🎯 Part 3: 计算处理效应

### 什么是 ATE？

**平均处理效应 (Average Treatment Effect, ATE)** 是所有人的 ITE 的平均值：

$$\text{ATE} = E[Y(1) - Y(0)] = E[\text{ITE}]$$

在我们的例子中，ATE 回答的问题是：

> "如果随机抽一个学生，**平均而言**，补习班会让 TA 的成绩提高多少分？"

### 两种计算 ATE 的方式

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def calculate_true_ate(df: pd.DataFrame) -> float:
    """计算真实 ATE（上帝视角）"""
    return df['ITE'].mean()


def calculate_naive_ate(df: pd.DataFrame) -> float:
    """计算朴素 ATE 估计（凡人视角）"""
    treated_mean = df[df['T'] == 1]['Y'].mean()
    control_mean = df[df['T'] == 0]['Y'].mean()
    return treated_mean - control_mean
```

**解释**：
- **真实 ATE**: 因为我们有上帝视角，知道所有人的 ITE，所以直接取平均即可
- **朴素估计**: 在现实中只能观测到实际的 Y，所以用处理组和对照组的平均差异来估计
- 在随机实验中，朴素估计是真实 ATE 的无偏估计，即 E[朴素估计] = 真实 ATE
</details>

In [ ]:
def calculate_true_ate(df: pd.DataFrame) -> float:
    """
    计算真实 ATE（上帝视角）
    
    因为我们知道所有人的 Y(1) 和 Y(0)，可以直接计算！
    
    公式: ATE = mean(Y(1) - Y(0)) = mean(ITE)
    """
    return df['ITE'].mean()


def calculate_naive_ate(df: pd.DataFrame) -> float:
    """
    计算朴素 ATE 估计（凡人视角）
    
    在现实中，我们不知道潜在结果，只能用观测数据：
    朴素估计 = mean(Y | T=1) - mean(Y | T=0)
    
    也就是：「参加补习的人的平均成绩」减去「没参加的人的平均成绩」
    """
    treated_mean = df[df['T'] == 1]['Y'].mean()
    control_mean = df[df['T'] == 0]['Y'].mean()
    return treated_mean - control_mean

In [ ]:
# 测试你的代码
if df['X'] is not None:
    true_ate = calculate_true_ate(df)
    naive_ate = calculate_naive_ate(df)
    
    if true_ate is not None and naive_ate is not None:
        print("📊 ATE 估计结果:")
        print(f"   真实 ATE（上帝视角）: {true_ate:.4f}")
        print(f"   朴素估计（凡人视角）: {naive_ate:.4f}")
        print(f"   偏差: {naive_ate - true_ate:.4f}")
        
        if abs(naive_ate - true_ate) < 1:
            print("\n🎉 太棒了！偏差很小！")
            print("   这是因为我们的处理 T 是完全随机分配的。")
        else:
            print("\n🤔 偏差有点大，让我们来看看为什么...")
    else:
        print("❌ 请完成 calculate_true_ate 和 calculate_naive_ate 函数！")

### 💡 思考时刻

如果你完成了上面的代码，你会发现：**朴素估计和真实 ATE 非常接近！**

这是为什么呢？🤔

因为在我们的数据生成过程中，**T 是完全随机分配的**（每个人 50% 概率参加补习班）。

在随机分配下：
- 参加补习的人和没参加的人，基础能力 X 的分布是一样的
- 所以他们之间的成绩差异，就是补习班的真实效果！

这就是**随机对照实验 (RCT)** 的威力！

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def counterfactual_analysis(df: pd.DataFrame) -> pd.DataFrame:
    """对每个个体，找出其观测值和反事实"""
    result = pd.DataFrame()
    
    # 添加处理状态
    result['参加补习'] = df['T'].map({1: '是 ✅', 0: '否 ❌'})
    
    # 添加观测结果
    result['观测成绩'] = df['Y']
    
    # 添加反事实结果
    result['反事实成绩'] = np.where(df['T'] == 1, df['Y0'], df['Y1'])
    
    # 添加个体处理效应
    result['补习效果(ITE)'] = df['ITE']
    
    return result
```

**解释**：
- **观测成绩**: 就是实际记录的成绩 Y
- **反事实成绩**: 使用 `np.where` 进行条件选择
  - 如果学生参加了补习(T=1)，反事实是没参加的成绩 Y0
  - 如果学生没参加(T=0)，反事实是参加了的成绩 Y1
- **ITE**: 两个潜在结果的差值，反映了补习对该学生的真实效果
- 这个表格展示了因果推断的核心困境：我们永远无法同时观测到两列
</details>

---

## 👀 Part 4: 反事实分析

让我们更深入地看看「反事实」是什么意思。

### 每个人的故事

In [ ]:
def counterfactual_analysis(df: pd.DataFrame) -> pd.DataFrame:
    """
    对每个个体，找出其观测值和反事实
    
    对于每个学生：
    - 如果 TA 参加了补习班(T=1)，我们观测到 Y(1)，反事实是 Y(0)
    - 如果 TA 没参加补习班(T=0)，我们观测到 Y(0)，反事实是 Y(1)
    """
    result = pd.DataFrame()
    
    # 添加处理状态
    result['参加补习'] = df['T'].map({1: '是 ✅', 0: '否 ❌'})
    
    # 添加观测结果
    result['观测成绩'] = df['Y']
    
    # 添加反事实结果
    # 如果 T=1，反事实是 Y0；如果 T=0，反事实是 Y1
    result['反事实成绩'] = np.where(df['T'] == 1, df['Y0'], df['Y1'])
    
    # 添加个体处理效应
    result['补习效果(ITE)'] = df['ITE']
    
    return result

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def compare_random_vs_confounded(n: int = 1000, seed: int = 42):
    """比较随机分配和混淆分配下的估计"""
    np.random.seed(seed)
    
    true_ate = 10.0
    X = np.random.randn(n)
    noise = np.random.randn(n) * 5
    Y0 = 50 + 5 * X + noise
    Y1 = 60 + 5 * X + noise
    
    # 场景 1: 随机分配
    T_random = np.random.binomial(1, 0.5, n)
    Y_random = np.where(T_random == 1, Y1, Y0)
    random_estimate = Y_random[T_random == 1].mean() - Y_random[T_random == 0].mean()
    
    # 场景 2: 混淆分配
    propensity = 1 / (1 + np.exp(-2 * X))
    T_confounded = np.random.binomial(1, propensity)
    Y_confounded = np.where(T_confounded == 1, Y1, Y0)
    confounded_estimate = Y_confounded[T_confounded == 1].mean() - Y_confounded[T_confounded == 0].mean()
    
    return true_ate, random_estimate, confounded_estimate, X, T_random, T_confounded
```

**解释**：
- **随机分配**: 每个人 50% 概率参加补习，T 独立于 X
- **混淆分配**: 使用 logistic 函数让 X 影响参加补习的概率
  - `propensity = 1 / (1 + exp(-2X))` 是标准的 logistic 函数
  - X 越大，propensity 越接近 1，即基础好的学生更可能参加补习
- **偏差产生机制**: 
  - 参加补习的学生 X 更高 → 他们的 Y0 和 Y1 都更高
  - 简单比较会把「X 高」的效果误认为是「补习」的效果
  - 这就是混淆偏差！
</details>

In [ ]:
# 测试你的代码
if df['X'] is not None:
    cf_df = counterfactual_analysis(df)
    
    if cf_df['观测成绩'] is not None:
        print("🔮 反事实分析结果（前10个学生）:")
        print("\n这个表格展示了每个学生的'另一种可能性'\n")
        display(cf_df.head(10))
        
        print("\n📝 解读:")
        print("   - '观测成绩' 是我们实际看到的")
        print("   - '反事实成绩' 是'如果 TA 做了不同选择会怎样'")
        print("   - 在现实中，我们永远无法知道反事实！")
    else:
        print("❌ 请完成 counterfactual_analysis 函数！")

---

## ⚠️ Part 5: 当随机化被破坏会怎样？

### 一个危险的场景

现在让我们看看一个更现实的情况：

> 学校没有随机分配补习班名额，而是**让学生自愿报名**。
> 
> 结果：**基础好的学生更愿意参加补习班**（因为他们更积极、更相信学习的价值）。

这时候会发生什么？

In [ ]:
def compare_random_vs_confounded(n: int = 1000, seed: int = 42):
    """
    比较随机分配和混淆分配下的估计
    
    场景 1: 随机分配 - 每个人 50% 概率参加补习班
    场景 2: 混淆分配 - 基础好的学生更可能参加补习班
    
    真实 ATE 都是 10 分！
    
    Returns:
        (true_ate, random_estimate, confounded_estimate, X, T_random, T_confounded)
    """
    np.random.seed(seed)
    
    # 真实 ATE（我们设定的）
    true_ate = 10.0
    
    # 生成基础能力
    X = np.random.randn(n)
    
    # 生成潜在结果
    noise = np.random.randn(n) * 5
    Y0 = 50 + 5 * X + noise  # 不参加补习
    Y1 = 60 + 5 * X + noise  # 参加补习（提高10分）
    
    # ============ 场景 1: 随机分配 ============
    # 完全随机分配（50% 概率）
    T_random = np.random.binomial(1, 0.5, n)
    
    # 观测结果
    Y_random = np.where(T_random == 1, Y1, Y0)
    
    # 计算朴素估计
    random_estimate = Y_random[T_random == 1].mean() - Y_random[T_random == 0].mean()
    
    # ============ 场景 2: 混淆分配 ============
    # 基础能力 X 越高，越可能参加补习班
    # 使用 logistic 函数: P(T=1|X) = 1 / (1 + exp(-2*X))
    
    # 计算每个人参加补习班的概率
    propensity = 1 / (1 + np.exp(-2 * X))
    
    # 根据概率分配处理
    T_confounded = np.random.binomial(1, propensity)
    
    # 观测结果
    Y_confounded = np.where(T_confounded == 1, Y1, Y0)
    
    # 计算朴素估计
    confounded_estimate = Y_confounded[T_confounded == 1].mean() - Y_confounded[T_confounded == 0].mean()
    
    return true_ate, random_estimate, confounded_estimate, X, T_random, T_confounded

In [ ]:
# 运行实验
results = compare_random_vs_confounded(n=2000)

if results[1] is not None:
    true_ate, random_est, confounded_est, X, T_random, T_confounded = results
    
    print("🔬 实验结果:")
    print("=" * 50)
    print(f"真实 ATE: {true_ate:.2f} 分")
    print(f"\n场景 1 - 随机分配:")
    print(f"   朴素估计: {random_est:.2f} 分")
    print(f"   偏差: {random_est - true_ate:+.2f} 分")
    
    print(f"\n场景 2 - 混淆分配（自愿报名）:")
    print(f"   朴素估计: {confounded_est:.2f} 分")
    print(f"   偏差: {confounded_est - true_ate:+.2f} 分")
    
    if confounded_est > true_ate + 2:
        print("\n⚠️ 发现问题！混淆分配下的估计偏高了！")
        print("   这是因为参加补习的学生本来就是基础好的，")
        print("   他们即使不参加补习成绩也会比较高！")
else:
    print("❌ 请完成 compare_random_vs_confounded 函数！")

In [ ]:
# 可视化混淆的影响
if results[1] is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 图1: 随机分配下 X 的分布
    ax1 = axes[0]
    ax1.hist(X[T_random==1], bins=30, alpha=0.5, label='参加补习', color='red', density=True)
    ax1.hist(X[T_random==0], bins=30, alpha=0.5, label='未参加', color='blue', density=True)
    ax1.set_xlabel('基础能力 X')
    ax1.set_ylabel('密度')
    ax1.set_title('场景1: 随机分配\n(两组的 X 分布相同)')
    ax1.legend()
    
    # 图2: 混淆分配下 X 的分布
    ax2 = axes[1]
    ax2.hist(X[T_confounded==1], bins=30, alpha=0.5, label='参加补习', color='red', density=True)
    ax2.hist(X[T_confounded==0], bins=30, alpha=0.5, label='未参加', color='blue', density=True)
    ax2.set_xlabel('基础能力 X')
    ax2.set_ylabel('密度')
    ax2.set_title('场景2: 自愿报名\n(参加者 X 更高！)')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 统计验证:")
    print(f"   随机分配 - 参加者平均 X: {X[T_random==1].mean():.3f}")
    print(f"   随机分配 - 未参加平均 X: {X[T_random==0].mean():.3f}")
    print(f"   混淆分配 - 参加者平均 X: {X[T_confounded==1].mean():.3f}")
    print(f"   混淆分配 - 未参加平均 X: {X[T_confounded==0].mean():.3f}")

### 🎓 关键洞察

从上面的实验中，我们发现了一个**非常重要**的结论：

| 分配方式 | 估计结果 | 为什么？ |
|---------|---------|--------|
| 随机分配 | ≈ 10 分（接近真实） | 两组的 X 分布相同，差异来自补习 |
| 自愿报名 | > 10 分（偏高！） | 参加者本来就更强，效果被夸大 |

这就是**混淆偏差 (Confounding Bias)** 的本质！

当存在一个变量 X（混淆变量）同时影响：
1. 是否接受处理 T（基础好 → 更愿意参加补习）
2. 结果 Y（基础好 → 成绩更高）

那么简单比较处理组和控制组的差异，就会得到有偏的结果！

---

## 📝 Part 6: 思考题

请在下面的 cell 中写下你的答案：

### 问题 1: 为什么我们无法同时观测到同一个人的 Y(0) 和 Y(1)?

**你的答案:**

*（在这里写下你的思考...）*

---

---

## 🎯 面试题模拟

### 概念题

**Q1: 什么是潜在结果框架（Potential Outcomes Framework）？它与传统回归分析有什么区别？**

<details>
<summary>答案</summary>

潜在结果框架是 Rubin 因果模型的核心，主要观点：

1. **核心思想**: 每个个体在接受处理和不接受处理时都有两个潜在结果 Y(1) 和 Y(0)
2. **基本问题**: 我们只能观测到其中一个，另一个是反事实（counterfactual）
3. **因果效应**: 定义为两个潜在结果的差异 ITE = Y(1) - Y(0)

**与回归的区别**：
- 回归关注**预测** E[Y|X]，因果推断关注**干预** E[Y(1) - Y(0)]
- 回归中 X 和 Y 的关系可能是相关性，因果推断要求因果性
- 回归假设所有变量都可观测，因果推断承认反事实不可观测

**例子**: 
- 回归：吃药的人康复率 80%，不吃药 60% → 预测下次吃药效果
- 因果：如果同一个人既吃又不吃会怎样？→ 药物真实效果
</details>

---

**Q2: 为什么随机对照实验（RCT）能够识别因果效应？**

<details>
<summary>答案</summary>

**核心原因**: 随机化使得处理分配 T 独立于潜在结果 {Y(0), Y(1)}

数学表达：$T \perp\!\!\!\perp \{Y(0), Y(1)\}$

**推导**：
$$E[Y|T=1] - E[Y|T=0] = E[Y(1)|T=1] - E[Y(0)|T=0]$$

由于随机化：
- $E[Y(1)|T=1] = E[Y(1)]$ （处理组和总体的 Y(1) 分布相同）
- $E[Y(0)|T=0] = E[Y(0)]$ （对照组和总体的 Y(0) 分布相同）

因此：
$$E[Y|T=1] - E[Y|T=0] = E[Y(1)] - E[Y(0)] = \text{ATE}$$

**直观理解**: 随机化让处理组和对照组在所有其他特征上都「平均」相同，唯一的区别就是是否接受处理。

**实践意义**: 
- A/B测试的理论基础
- 医学临床试验的金标准
- 但在很多场景下无法随机化（伦理、成本、可行性）
</details>

---

**Q3: 什么是 ATE、ATT、ATC？它们有什么区别？**

<details>
<summary>答案</summary>

三个重要的平均因果效应：

1. **ATE (Average Treatment Effect)**: 总体平均效应
   $$\text{ATE} = E[Y(1) - Y(0)]$$
   含义：随机抽一个人，平均而言处理的效果

2. **ATT (Average Treatment Effect on the Treated)**: 处理组的平均效应
   $$\text{ATT} = E[Y(1) - Y(0) | T=1]$$
   含义：对于实际接受处理的人，处理的平均效果

3. **ATC (Average Treatment Effect on the Control)**: 对照组的平均效应
   $$\text{ATC} = E[Y(1) - Y(0) | T=0]$$
   含义：对于实际未接受处理的人，如果给他们处理会有什么效果

**何时相等**: 在随机化下，ATE = ATT = ATC

**何时不等**: 当存在异质性效应（HTE）且处理分配非随机时

**实践选择**: 
- 政策评估：关注 ATT（对已参与者的效果）
- 推广决策：关注 ATE（如果全员推广的效果）
- 个性化：估计 CATE（条件平均效应）
</details>

---

### 编程题

**Q4: 请实现一个函数，模拟存在混淆的观测数据，并比较调整前后的估计偏差。**

要求：
1. 生成混淆变量 X 同时影响 T 和 Y
2. 真实 ATE = 5
3. 计算朴素估计和调整后估计
4. 返回两种估计及其偏差

<details>
<summary>参考代码</summary>

```python
def confounding_simulation(n=2000, true_ate=5.0, seed=42):
    """
    模拟混淆场景并比较估计方法
    
    DGP:
    - X ~ N(0, 1)
    - T ~ Bernoulli(sigmoid(2*X))  # X 影响 T
    - Y = 10 + true_ate*T + 3*X + ε  # X 和 T 都影响 Y
    """
    np.random.seed(seed)
    
    # 生成混淆变量
    X = np.random.randn(n)
    
    # T 受 X 影响
    propensity = 1 / (1 + np.exp(-2 * X))
    T = np.random.binomial(1, propensity)
    
    # Y 受 T 和 X 影响
    Y = 10 + true_ate * T + 3 * X + np.random.randn(n) * 0.5
    
    # 朴素估计（不控制 X）
    naive_ate = Y[T == 1].mean() - Y[T == 0].mean()
    naive_bias = naive_ate - true_ate
    
    # 调整估计（控制 X）
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(np.column_stack([T, X]), Y)
    adjusted_ate = model.coef_[0]
    adjusted_bias = adjusted_ate - true_ate
    
    return {
        'true_ate': true_ate,
        'naive_estimate': naive_ate,
        'naive_bias': naive_bias,
        'adjusted_estimate': adjusted_ate,
        'adjusted_bias': adjusted_bias
    }

# 测试
result = confounding_simulation()
print(f"真实 ATE: {result['true_ate']}")
print(f"朴素估计: {result['naive_estimate']:.4f} (偏差: {result['naive_bias']:+.4f})")
print(f"调整估计: {result['adjusted_estimate']:.4f} (偏差: {result['adjusted_bias']:+.4f})")
```

**输出示例**：
```
真实 ATE: 5.0
朴素估计: 11.0234 (偏差: +6.0234)  # 严重高估！
调整估计: 5.0123 (偏差: +0.0123)   # 接近真实值
```

**关键点**：
- 混淆导致朴素估计偏差 ≈ 6，因为 X→T 和 X→Y 都很强
- 控制 X 后，偏差几乎消失
- 这验证了混淆偏差公式: Bias = γ × δ ≈ 3 × 2 = 6
</details>

---

**Q5: 如何判断一个实验设计是否存在选择偏差？请写代码检验。**

<details>
<summary>参考代码</summary>

```python
def check_selection_bias(df, treatment_col='T', covariates=None):
    """
    检验处理组和对照组的协变量是否平衡
    
    方法：
    1. 比较两组的协变量均值差异
    2. 进行 t 检验
    3. 计算标准化均值差异 (SMD)
    
    返回：平衡性检验结果
    """
    if covariates is None:
        covariates = [col for col in df.columns if col != treatment_col]
    
    from scipy import stats
    
    results = []
    for cov in covariates:
        treated = df[df[treatment_col] == 1][cov]
        control = df[df[treatment_col] == 0][cov]
        
        # 均值差异
        mean_diff = treated.mean() - control.mean()
        
        # 标准化均值差异 (SMD)
        pooled_std = np.sqrt((treated.var() + control.var()) / 2)
        smd = mean_diff / pooled_std if pooled_std > 0 else 0
        
        # t 检验
        t_stat, p_value = stats.ttest_ind(treated, control)
        
        results.append({
            'variable': cov,
            'treated_mean': treated.mean(),
            'control_mean': control.mean(),
            'mean_diff': mean_diff,
            'SMD': smd,
            'p_value': p_value,
            'balanced': abs(smd) < 0.1  # SMD < 0.1 通常认为平衡
        })
    
    return pd.DataFrame(results)

# 示例使用
# 随机实验：应该平衡
df_rct = pd.DataFrame({
    'T': np.random.binomial(1, 0.5, 1000),
    'age': np.random.randn(1000),
    'income': np.random.randn(1000)
})

print("随机实验的平衡性检验：")
print(check_selection_bias(df_rct, 'T', ['age', 'income']))

# 观测数据：可能不平衡
X = np.random.randn(1000)
df_obs = pd.DataFrame({
    'T': (X + np.random.randn(1000) > 0).astype(int),  # T 受 X 影响
    'age': X,
    'income': X * 0.5 + np.random.randn(1000) * 0.5
})

print("\n观测数据的平衡性检验：")
print(check_selection_bias(df_obs, 'T', ['age', 'income']))
```

**判断标准**：
- SMD < 0.1: 良好平衡
- 0.1 ≤ SMD < 0.25: 可接受
- SMD ≥ 0.25: 严重不平衡，可能存在选择偏差
</details>

---

---

## 📐 数学推导

### 1. 观测结果公式

潜在结果和观测结果的关系是因果推断的基础：

$$Y_i = T_i \cdot Y_i(1) + (1 - T_i) \cdot Y_i(0)$$

**证明这个公式**：

由于 $T_i \in \{0, 1\}$，有两种情况：

- **情况 1**: $T_i = 1$ （接受处理）
  $$Y_i = 1 \cdot Y_i(1) + (1-1) \cdot Y_i(0) = Y_i(1)$$

- **情况 2**: $T_i = 0$ （未接受处理）
  $$Y_i = 0 \cdot Y_i(1) + (1-0) \cdot Y_i(0) = Y_i(0)$$

因此这个公式准确地描述了「我们观测到哪个潜在结果」的切换机制。

---

### 2. ATE 无偏性证明（随机化下）

**命题**: 在随机分配下，简单比较处理组和对照组的均值差异是 ATE 的无偏估计。

即证明：
$$E\left[\bar{Y}^{T=1} - \bar{Y}^{T=0}\right] = E[Y(1) - Y(0)] = \text{ATE}$$

**证明**：

处理组的期望结果：
$$E[Y | T=1] = E[Y(1) | T=1]$$

在随机化下，$T \perp\!\!\!\perp \{Y(0), Y(1)\}$（处理独立于潜在结果），因此：
$$E[Y(1) | T=1] = E[Y(1)]$$

同理，对照组的期望结果：
$$E[Y | T=0] = E[Y(0) | T=0] = E[Y(0)]$$

因此：
$$E[Y | T=1] - E[Y | T=0] = E[Y(1)] - E[Y(0)] = \text{ATE}$$

**关键洞察**: 随机化保证了 $T$ 独立于潜在结果，使得处理组和对照组是「可比的」（comparable）。

---

### 3. 因果推断的基本问题

**基本问题**: 对于任何个体 $i$，我们永远无法同时观测到 $Y_i(1)$ 和 $Y_i(0)$。

数学表达：
$$\text{Missing Data Problem: } \begin{cases}
Y_i(1) \text{ 可观测} & \text{if } T_i = 1 \\
Y_i(0) \text{ 可观测} & \text{if } T_i = 0
\end{cases}$$

这导致：
- ITE $= Y_i(1) - Y_i(0)$ 永远无法在个体层面精确计算
- 我们只能估计群体层面的 ATE $= E[Y(1) - Y(0)]$

**反事实不可观测性** 是因果推断区别于预测问题的根本特征。

---

### 4. 选择偏差公式

当处理分配不是随机时，朴素估计会有偏差：

$$\begin{align}
E[Y|T=1] - E[Y|T=0] &= E[Y(1)|T=1] - E[Y(0)|T=0] \\
&= E[Y(1)|T=1] - E[Y(0)|T=1] \\
&\quad + E[Y(0)|T=1] - E[Y(0)|T=0] \\
&= \underbrace{E[Y(1) - Y(0) | T=1]}_{\text{处理组的 ATT}} + \underbrace{E[Y(0)|T=1] - E[Y(0)|T=0]}_{\text{选择偏差}}
\end{align}$$

**第二项就是选择偏差**：即使不接受处理，处理组和对照组的结果也会不同。

在随机化下，$E[Y(0)|T=1] = E[Y(0)|T=0]$，偏差项为 0。

---

### 问题 2: 随机实验 (RCT) 为什么能识别 ATE?

**提示:** 想想随机分配对处理组和控制组有什么影响？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 在混淆分配的场景中，朴素估计是偏高还是偏低？为什么？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: ITE（个体处理效应）在实践中为什么重要？举一个例子。

**提示:** 想想个性化营销、精准医疗等场景...

**你的答案:**

*（在这里写下你的思考...）*

---

---

## 🎉 总结

恭喜你完成了第一个练习！让我们回顾一下学到的核心概念：

### 核心公式

| 概念 | 公式 | 含义 |
|-----|------|------|
| 潜在结果 | $Y(0), Y(1)$ | 两种可能的世界 |
| 观测结果 | $Y = T \cdot Y(1) + (1-T) \cdot Y(0)$ | 我们实际看到的 |
| ITE | $\text{ITE}_i = Y_i(1) - Y_i(0)$ | 个体处理效应 |
| ATE | $\text{ATE} = E[Y(1) - Y(0)]$ | 平均处理效应 |

### 关键洞察

1. **因果推断的根本问题**: 我们永远无法同时观测到两个潜在结果
2. **随机实验的威力**: 随机分配让两组的特征分布相同，消除混淆
3. **混淆偏差**: 当某些变量同时影响处理和结果时，简单比较会出错

### 下一步

既然我们理解了问题的本质，下一个练习我们将学习**因果图 (DAG)**，用图形化的方式来表示和分析因果关系！

---

**「相关不等于因果」不是一句空话，而是这整个领域存在的理由。**